# 最終課題

### 課題内容
『武蔵野大学のWebサイトのサイトマップを抽出する』
・武蔵野大学Webサイトのトップページにアクセス
・同一ドメインの全てのリンク（コメントアウトされていないもの）を辿り，全ページのURLと<title>を辞書型変数に格納する
key：URL
value：<title></title>で挟まれた文字列
・辞書型変数を print() で表示する


・注意
Webサイトにアクセスする際は必ず time.sleep(秒数) を使って負荷軽減をすること
課題用のipynb形式のファイルを作成し，githubフローでファイルの変更履歴を管理すること


In [29]:
#　インポート
import requests
from bs4 import BeautifulSoup
import time

In [30]:
url = "https://www.musashino-u.ac.jp/"
# クロール対象をこのドメイン内に限定するための文字列。
TARGET_DOMAIN = "musashino-u.ac.jp"
# 全ページのurlを取得するため、これから訪問すべきURLを管理するためのリストを準備する。
# 最初は起点となるBASE_URLだけを入れておく。
urls_to_visit = [url]
sitemap = {}    # サイトマップ用の辞書を作成


In [31]:
res = requests.get(url)
time.sleep(0.5) # アクセス負荷軽減
soup = BeautifulSoup(res.text, 'html.parser')


In [32]:
# --- 3. クロール処理のメインループ ---
print("ステップ3: サイト全体の巡回を開始します。")
# 取得したページの文字コードを自動判別して設定する（文字化けを防ぐため）
res.encoding = res.apparent_encoding

# 訪問予定リスト(urls_to_visit)が空になるまで、このループを繰り返す。
while urls_to_visit:
    # リストの先頭からURLを1つ取り出す。
    current_url = urls_to_visit.pop(0)

    # もし取り出したURLが既に訪問済み(sitemapにキーが存在する)なら、
    # この回のループはここで中断し、次のURLの処理に移る。
    if current_url in sitemap:
        continue
    else:   
        print(f"処理中: {current_url}")
        time.sleep(1)
        res = requests.get(current_url, timeout=10) # 10秒応答がなければタイムアウト
        res.encoding = res.apparent_encoding

        # 取得したコンテンツがHTMLページでなければスキップする。
        if 'text/html' not in res.headers.get('content-type', ''):
            print(" HTMLではないためスキップします。")
            sitemap[current_url] = "HTML以外のコンテンツ"
            continue

    # --- ページタイトルの取得と格納 ---
    # soup.find('title')を使って、HTMLの中から<title>タグを1つ見つけ出す。
    title_tag = soup.find('title')
    page_title = title_tag.text.strip() if title_tag else "タイトルなし"  # タイトルタグがない場合の対策
    sitemap[current_url] = page_title    # 訪問済みの印として、URLとタイトルをsitemap辞書に格納する。

    # --- ページ内の全リンクを抽出し、訪問予定リストに追加 ---
    # soup.find_all('a', href=True)を使い、href属性を持つ<a>タグを全て抽出する。
    a_tags = soup.find_all('a', href=True)


    for a_tag in a_tags:
        # a_tag.get('href')で、<a>タグのhref属性の値（リンク先URL）を取得する。
        href = a_tag.get('href')

        # --- URLの形式を整える処理 ---
        # hrefが'/'で始まる場合（例: /faculty/）、ドメイン名と結合して完全なURLにする。
        if href.startswith('/'):
            new_url = "https://www.musashino-u.ac.jp" + href
        # hrefが'http'で始まる場合、それは既に完全なURLなのでそのまま使う。
        elif href.startswith('http'):
            new_url = href
        # それ以外（例: #top, javascript:void(0)など）は処理対象外とする。
        else:
            continue

        # --- 訪問予定リストに追加するかの判定 ---
            # 以下の3つの条件を全て満たすURLのみをリストに追加する。
            # 1. URLにターゲットドメインの文字列が含まれているか。
            # 2. そのURLがまだ訪問済みリスト(sitemap)に入っていないか。
            # 3. そのURLが既に訪問予定リスト(urls_to_visit)に入っていないか。
        if TARGET_DOMAIN in new_url and new_url not in sitemap and new_url not in urls_to_visit:
                urls_to_visit.append(new_url)

# --- 6. 最終結果の表示 ---
print("\n" + "="*50)
print("クロールが完了しました。")
print("="*50 + "\n")

# 課題の要件である、完成した辞書型変数をprint()で表示する。
print("--- サイトマップ（辞書型変数） ---")
print(sitemap)

print(f"\n合計 {len(sitemap)} ページの情報を収集しました。")
    







ステップ3: サイト全体の巡回を開始します。
処理中: https://www.musashino-u.ac.jp/
処理中: https://ef.musashino-u.ac.jp/donation/
処理中: https://www.musashino-u.ac.jp/access.html
処理中: https://www.musashino-u.ac.jp/admission/request.html
処理中: https://www.musashino-u.ac.jp/contact.html
処理中: https://www.musashino-u.ac.jp/prospective-students.html
処理中: https://www.musashino-u.ac.jp/students.html
処理中: https://www.musashino-u.ac.jp/alumni.html
処理中: https://www.musashino-u.ac.jp/parents.html
処理中: https://www.musashino-u.ac.jp/business.html
処理中: https://www.musashino-u.ac.jp/guide/
処理中: https://www.musashino-u.ac.jp/guide/profile/
処理中: https://www.musashino-u.ac.jp/guide/activities/
処理中: https://www.musashino-u.ac.jp/guide/campus/
処理中: https://www.musashino-u.ac.jp/guide/facility/
処理中: https://www.musashino-u.ac.jp/guide/information/
処理中: https://www.musashino-u.ac.jp/guide/profile/media/
処理中: https://www.musashino-u.ac.jp/admission/
処理中: https://www.musashino-u.ac.jp/admission/faculty/
処理中: https://www.musashino-u.ac.jp/